# Testing

In [31]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import joblib

import getScreenTime

## get data

In [32]:
data = getScreenTime.get_screen_time_data(758000000, 758935000) # jan 7 10pm - January 18, January 18, 2025 6:16:40 PM


('com.google.Chrome', (2025, 1, 22, 33, 20, 2))
('off', (2025, 1, 22, 58, 13, 2))
('off', (2025, 1, 23, 26, 6, 2))
('com.google.Chrome', (2025, 1, 23, 58, 30, 2))
('off', (2025, 1, 0, 23, 54, 3))
('com.google.Chrome', (2025, 1, 0, 44, 17, 3))
('off', (2025, 1, 1, 18, 38, 3))
('off', (2025, 1, 1, 46, 1, 3))
('off', (2025, 1, 2, 25, 53, 3))
('off', (2025, 1, 3, 3, 13, 3))
('off', (2025, 1, 3, 24, 45, 3))
('off', (2025, 1, 4, 4, 34, 3))
('off', (2025, 1, 4, 41, 23, 3))
('off', (2025, 1, 5, 14, 46, 3))
('off', (2025, 1, 5, 45, 54, 3))
('off', (2025, 1, 6, 17, 6, 3))
('off', (2025, 1, 6, 49, 29, 3))
('off', (2025, 1, 7, 10, 34, 3))
('off', (2025, 1, 7, 32, 42, 3))
('off', (2025, 1, 7, 55, 24, 3))
('off', (2025, 1, 8, 22, 31, 3))
('off', (2025, 1, 8, 52, 50, 3))
('off', (2025, 1, 9, 30, 38, 3))
('off', (2025, 1, 9, 59, 30, 3))
('off', (2025, 1, 10, 27, 56, 3))
('off', (2025, 1, 11, 1, 3, 3))
('off', (2025, 1, 11, 38, 3, 3))
('off', (2025, 1, 12, 9, 18, 3))
('off', (2025, 1, 12, 34, 56, 3))
(

## model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),  
    tf.keras.layers.Dense(20, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])